In [1]:
import numpy as np
import pandas
import pm4py
from matplotlib import pyplot as plt
from sklearn.mixture import GaussianMixture
import scipy.stats as stats
import ot
import os
from tqdm import tqdm
import collections
import matplotlib.dates as md
import importlib
import pickle
import random
import CRPS.CRPS as pscore


pandas.set_option('display.max_columns', None)
#pandas.set_option('display.max_rows', None)


import sys
sys.path.append('../../TaskExecutionTimeMining/')
from drbart_parser import *
from event_log_transformer import *

#sys.path.append('../../Evaluation')
sys.path.append('../../Evaluation/')
from PCR_evaluation.dumas_evaluation import *

In [2]:
with open('../transformed_event_logs/PCR_start_end_test.pickle', 'rb') as f:
    test_event_log = pickle.load(f)

/tmp/ipykernel_1061740/2720744415.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  test_event_log = pickle.load(f)


In [3]:
with open('../../../models/pcr_dumas/dumas_model.pickle', 'rb') as f:
    dumas_model = pickle.load(f)

In [4]:
import conduct_evaluation
#sample_dumas = SampleOutcomes_Dumas_Normal(test_event_log, dumas_model)

evaluator = conduct_evaluation.ConductEvaluation(dumas_model, SampleOutcomes_Dumas_Normal, {},
                                     test_event_log, n=10000)

In [ ]:
likelihoods = evaluator.sample_cases(plot_cases=False, multiprocessing=False)

  6%|███████▋                                                                                                                           | 72/1235 [13:47<3:34:11, 11.05s/it]

In [ ]:
exclude = list(filter(lambda k : k[1]==0, likelihoods[0].items()))

In [ ]:
exclude

In [ ]:
np.sum(np.log(np.array(list(likelihoods[0].values()))))

In [ ]:
get_pscores = lambda likelihoods : [pscore(likelihoods[1][i], likelihoods[2][k][3]).compute()[0] for i, k in enumerate(list(likelihoods[0].keys()))]

np.mean(get_pscores(likelihoods))